# 单元版
- nn.RNNCell
- nn.LSTMCell
- nn.GRUCell  
***
``` python
rnn = nn.RNNCell(input_size=10, hidden_size=20, bias=True, nonlinearity='tanh')
input = torch.randn(6, 3, 10)
hx = torch.randn(3, 20)
output = []
for i in range(input.size(0)):
    hx = rnn(input[i], hx) # 输出只有隐含状态的输出
    output.append(hx)

```
***

# 封装版
- nn.RNN
- nn.LSTM
- nn.GRU 
***
``` python
rnn = nn.RNN(input_size=10, hidden_size=20, num_layers=2, nonlinearity='tanh', bias=True, batch_first=False, dropout=0,bidirectional=False) 
input = torch.rand(5, 3, 10) # 输入的形状为（seq_len, batch, feature） 序列长度为5， batchsize为3，特征数量为10  
h0 = torch.randn(2, 3, 20) # h0的形状为（num_layers*num_direction, batch, hidden_size）  
output, hn = rnn(input, h0) # output的形状为（seq_len，batch, num_directions*hidden_size）, hn的形状为（num_layers*num_directions, batch,hidden_size）  
```
***
注：两者的最大区别是输入不同，单元版的输入是一个时刻的输入，而封装版的输入是一个时间序列的输入（包含序列长度）。因此如果输入的长度不一样时，可以采用单元版模块，在训练的时候使用for循环得到最终的输出；当输入长度一样时，可以采用封装版。

# 词嵌入
## 表示方法
### one-hot表示
独热表示的向量长度为词典的大小，向量的分量只有一个1.其他全为0,1的位置对应该词在词典中的位置。
### 词袋模型
one-hot是用来表示一个字或者词的，而词袋模型是用来表示一个句子或者一篇文档的，其向量长度也为字典长度，其中每个元素为字典中某个词在该句子会文档中出现的频率。  
### TF-IDF  
词频-逆文档频率
$$TF_{ij}=\frac{单词i在文档j中出现的次数}{文档j包含的总的单词数量}$$  
$$IDF_i=log(\frac{语料库中文档总数}{语料库中包含单词i的文档数+1})$$  
$$TF-IDF = TF * IDF$$
### 分布式表示word2vec
词袋模型与TF-IDF表示方法只考虑了单词的词频，而没有考虑单词与单词之间的关系，即任意两个词之间都是独立的。为了克服这个缺点，提出了单词的分布式表示方法，其中最著名的为word2vec。  
分布式表示的优点是解决了词汇与位置无关的问题，缺点是学习过程相对复杂且受训练语料的影响很大。

# LSTM实现词性判别
每个单词都有词性，具体表示哪种词性与单词所处的上下文相关，这种上下文关系可以使用LSTM来进行建模。

In [68]:
# 定义训练数据,2条
train_data = [('The cat ate the fish'.split(), ['DET', 'NN', 'V', 'DET', 'NN']), 
              ('They read that book'.split(), ['NN', 'V', 'DET', 'NN'])]
# 定义测试数据，1条
test_data = [('They ate the fish'.split())]

In [69]:
# 构建单词的索引字典
word_to_idx = {}
for sent, tags in train_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)
# 构建词性索引字典
tag_to_idx = {'DET':0, 'NN':1, 'V':2}
print(tag_to_idx)

{'The': 0, 'cat': 1, 'ate': 2, 'the': 3, 'fish': 4, 'They': 5, 'read': 6, 'that': 7, 'book': 8}
{'DET': 0, 'NN': 1, 'V': 2}


In [75]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 构建网络，一个Embedding层，一个LSTM层，一个全连接层
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(100)
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # 将一个整数变成一个向量，其实就是一个全连接层，返回的词嵌入
                                                                 # 为全连接层的weight,
                                                                # 只不过这个全连接层的输出紧接着输入到LSTM层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, output_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        """
            初始化cell_state和hidden_state
        """
        return (torch.zeros(1, 1, self.hidden_dim).to(device),
               torch.zeros(1, 1, self.hidden_dim).to(device))
    def forward(self, sent):
        """
            sent为句子的索引列表
        """
        # 获得词嵌入矩阵
        embeds = self.embedding(sent)
#         print(embeds)
#         print(self.embedding.weight)
        # 改为Lstm的输入格式
        embeds = embeds.view(len(sent), 1, -1)
        lstm_out, hn = self.lstm(embeds, self.hidden) # lstm_out形状为（seq_len, batch, hidden_dim）
        # 修改lstm_out的形状，作为全连接层的输入
        tag_space = self.fc1(lstm_out.view(len(sent), -1)) 
#         print(tag_space.shape)
        # 计算每个单词属于各个词性的概率
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
        

In [76]:
def prepare_sent(sent, to_idx):
    return torch.LongTensor([to_idx[i] for i in sent])

EPOCH = 100
embedding_dim = 10
hidden_dim = 5

model = LSTMTagger(embedding_dim, hidden_dim, len(word_to_idx), len(tag_to_idx))
model = model.cuda()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(EPOCH):
    model.train()
    for sent, tags in train_data:
        inputs = prepare_sent(sent, word_to_idx)
        targets = prepare_sent(tags, tag_to_idx)
        inputs = inputs.cuda()
        targets = targets.cuda()
        # 前面传播
        output = model(inputs)
        loss = criterion(output, targets)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        # 优化
        optimizer.step()
    
# 测试
model.eval()
for sent in test_data:
    inputs = prepare_sent(sent, word_to_idx)
    inputs = inputs.cuda()
    outputs = model(inputs)
#     _, pred = outputs.topk(1, dim=1)
    _, pred = torch.max(outputs, dim=1)

print(tag_to_idx)
print(test_data)
res = []
for p in pred:
    for k, v in tag_to_idx.items():
        if p == v:
            res.append(k)
            break
print(res)


{'DET': 0, 'NN': 1, 'V': 2}
[['They', 'ate', 'the', 'fish']]
['NN', 'V', 'DET', 'NN']


#  LSTM预测股票行情